# Extract Trajectories (No normalization)

In [ ]:
import pandas as pd

def format_data(file_path):
    # Import and parse data
    pedestrian_data = pd.read_csv(file_path, sep='\t', header=None)
    pedestrian_data.columns = ["Frame Number", "Pedestrian ID", "X-Coordinate", "Y-Coordinate"]

    # Group the data by 'Pedestrian ID' and extract trajectories as lists of (x, y) tuples
    trajectories = pedestrian_data.groupby("Pedestrian ID").apply(lambda group: (group.name, list(zip(group["X-Coordinate"], group["Y-Coordinate"]))))
    trajectories = trajectories.tolist()

    # Return list of trajectories
    # Each entry is (Pedestrian ID, Trajectory)
    return trajectories

for trajectory in format_data(r'src\datasets\eth\test\biwi_eth.txt'):
    print(trajectory)

(1.0, [(8.46, 3.59), (9.57, 3.79), (10.67, 3.99), (11.73, 4.32), (12.81, 4.61)])
(2.0, [(13.64, 5.8), (12.09, 5.75), (11.37, 5.8), (10.31, 5.97), (9.57, 6.24), (8.73, 6.34), (7.94, 6.5), (7.17, 6.62), (6.47, 6.68), (5.86, 6.82), (5.24, 6.98), (4.87, 7.16), (4.51, 7.58), (4.2, 7.3), (3.95, 7.71), (3.47, 7.86), (2.82, 8.0), (2.01, 8.0), (1.28, 7.82), (0.54, 7.4), (-0.18, 7.06), (-0.83, 6.43), (-1.52, 6.05)])
(3.0, [(12.49, 6.6), (11.94, 6.77), (11.03, 6.84), (10.21, 6.81), (9.36, 6.85), (8.59, 6.85), (7.78, 6.84), (6.96, 6.84), (6.29, 7.0), (5.62, 7.1), (5.06, 7.04), (4.69, 7.0), (4.35, 7.01), (3.76, 6.99), (3.19, 6.89), (2.62, 7.13), (1.78, 7.15), (1.01, 6.96), (0.07, 6.91), (-0.72, 6.66)])
(4.0, [(-1.32, 5.11), (-0.33, 5.06), (0.76, 5.0), (1.76, 4.89), (2.81, 4.83), (3.8, 4.77), (4.97, 4.69), (5.97, 4.7), (6.97, 4.67), (8.12, 4.95), (9.11, 5.01), (10.12, 5.13), (11.01, 5.32), (11.93, 5.47)])
(5.0, [(-1.48, 4.43), (-0.49, 4.38), (0.6, 4.23), (1.6, 4.13), (2.67, 4.07), (3.7, 4.02), (4.75

# Extract Traectories (Z-scale normalization)

In [2]:
import pandas as pd

def format_data(file_path):
    # Import and parse data
    pedestrian_data = pd.read_csv(file_path, sep='\t', header=None)
    pedestrian_data.columns = ["Frame Number", "Pedestrian ID", "X-Coordinate", "Y-Coordinate"]

    # Calculate mean and standard deviation for normalization
    x_mean = pedestrian_data["X-Coordinate"].mean()
    x_std = pedestrian_data["X-Coordinate"].std()
    y_mean = pedestrian_data["Y-Coordinate"].mean()
    y_std = pedestrian_data["Y-Coordinate"].std()

    # Apply z-scale normalization
    pedestrian_data["X-Coordinate"] = (pedestrian_data["X-Coordinate"] - x_mean) / x_std
    pedestrian_data["Y-Coordinate"] = (pedestrian_data["Y-Coordinate"] - y_mean) / y_std

    # Group the data by 'Pedestrian ID' and extract normalized trajectories
    trajectories = pedestrian_data.groupby("Pedestrian ID").apply(lambda group: (group.name, list(zip(group["X-Coordinate"], group["Y-Coordinate"]))))
    trajectories = trajectories.tolist()

    # Return both the normalized trajectories and the values needed for inverse transformation
    return trajectories, x_mean, x_std, y_mean, y_std

def denormalize_trajectory(normalized_trajectory, x_mean, x_std, y_mean, y_std):
    # Convert each normalized (x, y) back to original coordinates
    denormalized_trajectory = [(x * x_std + x_mean, y * y_std + y_mean) for x, y in normalized_trajectory]
    return denormalized_trajectory

# Example usage
normalized_trajectories, x_mean, x_std, y_mean, y_std = format_data(r'src\datasets\eth\test\biwi_eth.txt')

# Denormalize each trajectory
for pedestrian_id, normalized_trajectory in normalized_trajectories:
    original_trajectory = denormalize_trajectory(normalized_trajectory, x_mean, x_std, y_mean, y_std)
    print(f"Pedestrian ID: {pedestrian_id}, Original Trajectory: {original_trajectory}")


Pedestrian ID: 1.0, Original Trajectory: [(8.46, 3.59), (9.57, 3.79), (10.67, 3.99), (11.73, 4.32), (12.81, 4.61)]
Pedestrian ID: 2.0, Original Trajectory: [(13.64, 5.8), (12.09, 5.75), (11.37, 5.8), (10.310000000000002, 5.97), (9.57, 6.24), (8.73, 6.34), (7.94, 6.5), (7.17, 6.62), (6.47, 6.68), (5.86, 6.82), (5.24, 6.98), (4.87, 7.16), (4.51, 7.58), (4.2, 7.3), (3.95, 7.71), (3.47, 7.86), (2.82, 8.0), (2.01, 8.0), (1.2800000000000002, 7.82), (0.54, 7.4), (-0.17999999999999972, 7.06), (-0.8300000000000001, 6.43), (-1.5200000000000005, 6.05)]
Pedestrian ID: 3.0, Original Trajectory: [(12.49, 6.6), (11.94, 6.77), (11.03, 6.84), (10.21, 6.81), (9.36, 6.85), (8.59, 6.85), (7.78, 6.84), (6.96, 6.84), (6.29, 7.0), (5.62, 7.1), (5.06, 7.04), (4.69, 7.0), (4.35, 7.01), (3.76, 6.99), (3.19, 6.89), (2.62, 7.13), (1.7800000000000002, 7.15), (1.0099999999999998, 6.96), (0.07000000000000028, 6.91), (-0.7199999999999998, 6.66)]
Pedestrian ID: 4.0, Original Trajectory: [(-1.3200000000000003, 5.11), (